## Example of Measurement Combination 

As example to show the combination of different measurements and why it is important to consider their correlations, we create this example: a fitting to 6 tracker elements. 

A tracker has detector 6 elements at x = -11, -10, -9, +9, +10 and +11 cms, which each measure a track’s y-coordinate to an accuracy of *± 1* cm. A straight line $y = a + bx$ is fitted (for example by chi-squared) to the data.
We fit 
- the 3 elements at negative x (line L1); 
- the 3 elements at positive  x (L2); 
- all 6 detector elements.

We study how one can combine the result from the first two fits (L1 and L2)  for the *a* and *b* parameters. 

In [1]:
double xpoints[] = { -11,-10,-9, 9,10,11};

In [2]:
auto f1 = new TF1("f1","[A]*x + [B]");
f1->SetParameter("A",0.5);
f1->SetParameter("B",0.);

In [3]:
double ypoints[6]; 
double sigma = 1; 
TRandomMixMax r(1);
ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");

In [4]:
for (int i = 0; i < 6; ++i) { 
    ypoints[i] = f1->Eval(xpoints[i]) + r.Gaus(sigma);
}

In [5]:
auto gr = new TGraphErrors(6,xpoints,ypoints);
for (int i = 0; i < 6; ++i)
    gr->SetPointError(i,0,1);


In [6]:
%jsroot on

In [7]:
gr->SetMarkerStyle(20);
gr->Draw("AP");
gPad->Draw();

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


#### 1. Straight Line Fit to Negative Points (L1)

We perform a straight line fit to the 3 negative points 
and we print the result and the correlation matrix obtained from the fit.

In [8]:
f1->SetRange(-12,0); 
f1->SetLineColor(kBlue);
r1 = gr->Fit(f1,"S R +");
r1->GetCorrelationMatrix().Print();
gPad->Draw();


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      4.56746
NDf                       =            1
Edm                       =  5.58334e-21
NCalls                    =           35
A                         =  -0.00840201   +/-   0.707107    
B                         =      -3.9074   +/-   7.0946      

2x2 matrix is as follows

     |      0    |      1    |
-------------------------------
   0 |          1      0.9967 
   1 |     0.9967           1 



#### 2. Straight Line Fit to Positive Points (L2)


In [9]:
f1->SetRange(0,12);
f1->SetLineColor(kBlack);
r2 = gr->Fit(f1,"S R +");
r2->GetCorrelationMatrix().Print();
gPad->Draw();


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =     0.312572
NDf                       =            1
Edm                       =   1.9506e-22
NCalls                    =           39
A                         =    -0.123414   +/-   0.707107    
B                         =      8.43832   +/-   7.0946      

2x2 matrix is as follows

     |      0    |      1    |
-------------------------------
   0 |          1     -0.9967 
   1 |    -0.9967           1 



#### 3. Straight Line Fit to All Points (L3)

In [10]:
f1->SetRange(-12,12);
f1->SetLineColor(kRed);
r3 = gr->Fit(f1,"S  +");
r3->GetCorrelationMatrix().Print();


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      6.40734
NDf                       =            4
Edm                       =  1.05151e-17
NCalls                    =           28
A                         =     0.547291   +/-   0.0406894   
B                         =       1.6904   +/-   0.408248    

2x2 matrix is as follows

     |      0    |      1    |
-------------------------------
   0 |          1  -7.544e-10 
   1 | -7.544e-10           1 



In [11]:
gr->SetMarkerStyle(20);
gr->Draw("AP"); gPad->Draw();

### Combination of the obtained results

We perform now the combination of the parameters for the first two fit and we compare the combined result with the combined fit.
We do the combination using first just as weight the inverse of the squared error
$$w_i = \frac{ 1/\sigma_{i}^2 }{\sum_i 1/\sigma_{i}^2 }$$

In [12]:
ipar = 0;
v1 = r1->Parameter(ipar);
err1 = r1->Error(ipar);
v2 = r2->Parameter(ipar);
err2 = r2->Error(ipar);


In [13]:
w1  = 1./(err1*err1);
w2 = 1./(err2*err2);
wtot = w1+w2; 
value_comb = (w1*v1 + w2*v2)/wtot; 
err_comb = sqrt(1./wtot);

In [14]:
std::cout << "The combined value for parameter  " << r1->ParName(ipar) << " is : " 
         << value_comb << " +/- " << err_comb << std::endl;

The combined value for parameter  A is : -0.0659078 +/- 0.5


As you see these values do not make sense, because we did not take into account the correlations between the two fitted parameters. We cannot consider them indipendently. 
If we now we do the combination using instead the correct weights based on the inverse of the covariance (error) matrices.

$$W_i =  M_{i}^{-1} $$

where $M_i$ is the covariance matrix for the $i$ measurement, and $W_i$ is the resulting un-normalized weight matrix. 

In [15]:
M1 = r1->GetCovarianceMatrix();
M2 = r2->GetCovarianceMatrix();
W1 = M1.Invert();
W2 = M2.Invert(); 

Using simple matrix algebra we can get then the combined values:
$$X_{comb} = (W_1+W_2)^{-1} * ( W_1 * X_1 + W_2 * X_2 ) $$

Where $X_1$ and $X_2$ are the vectors of the measurements. For this example $X_1 = (a_1,b_1)$ 

The combined error will be instead
    $$M_{comb} = (W_1+W_2)^{-1}$$
    
We can compute first the combined error:     

In [16]:
Wcomb = W1+W2; 
Mcomb = Wcomb.Invert();
Mcomb.Print();


2x2 matrix is as follows

     |      0    |      1    |
-------------------------------
   0 |   0.001656   5.333e-12 
   1 |  5.333e-12      0.1667 



In [17]:
std::cout << "Combined error on A = " << sqrt(Mcomb(0,0)) << std::endl;
std::cout << "Combined error on B = " << sqrt(Mcomb(1,1)) << std::endl;

Combined error on A = 0.0406894
Combined error on B = 0.408248


And then the combined value

In [18]:
TVectorD Vec1(2,r1->GetParams());
TVectorD Vec2(2,r2->GetParams());

In [19]:
TVectorD Vec_comb(2);
Vec_comb = W1*Vec1+W2*Vec2;
Vec_comb = Mcomb * Vec_comb; 
Vec_comb.Print();


Vector (2)  is as follows

     |        1  |
------------------
   0 |0.547291 
   1 |1.6904 



In [20]:
std::cout << "Computed combined values are : \n";
std::cout << "A = " << Vec_comb(0) << " +/- " << sqrt(Mcomb(0,0)) << std::endl;
std::cout << "B = " << Vec_comb(1) << " +/- " << sqrt(Mcomb(1,1)) << std::endl;

Computed combined values are : 
A = 0.547291 +/- 0.0406894
B = 1.6904 +/- 0.408248


In [21]:
std::cout << "Values obtained from the combined fit are : \n";
std::cout << "A = " << r3->Parameter(0) << " +/- " << r3->Error(0) << std::endl;
std::cout << "B = " << r3->Parameter(1) << " +/- " << r3->Error(1) << std::endl;

Values obtained from the combined fit are : 
A = 0.547291 +/- 0.0406894
B = 1.6904 +/- 0.408248


As you see the direct computation or the one obtained from the fit give exactly the same result as expected.